<a href="https://colab.research.google.com/github/FuadBabaev/Advanced_RL/blob/main/homework_4/hw-4-advantage-actor-critic.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Implementing Advantage-Actor Critic (A2C) - 2 pts

In this notebook you will implement Advantage Actor Critic algorithm that trains on a batch of Atari 2600 environments running in parallel.

Firstly, we will use environment wrappers implemented in file `atari_wrappers.py`. These wrappers preprocess observations (resize, grayscal, take max between frames, skip frames, stack them together, prepares for PyTorch and normalizes to [0, 1]) and rewards. Some of the wrappers help to reset the environment and pass `done` flag equal to `True` when agent dies.
File `env_batch.py` includes implementation of `ParallelEnvBatch` class that allows to run multiple environments in parallel. To create an environment we can use `nature_dqn_env` function.

In [1]:
!pip install gymnasium==1.0.0
!pip install ale-py==0.10.2
!pip install opencv-python
!pip install gymnasium[other]

In [2]:
import os
os.chdir('/content')

In [3]:
!pip install tensorboardX

In [4]:
import numpy as np
from atari_wrappers import nature_dqn_env
import gymnasium as gym
from atari_wrappers import TensorboardSummaries

nenvs = 8    # change this if you have more than 8 CPU ;)
env = gym.vector.AsyncVectorEnv([lambda: nature_dqn_env("SpaceInvadersNoFrameskip-v4") for _ in range(nenvs)])
env = TensorboardSummaries(env, "spaceinvaders")


n_actions = env.single_action_space.n
obs, info = env.reset()
assert obs.shape == (nenvs, 4, 84, 84)
assert obs.dtype == np.float32

Next, we will need to implement a model that predicts logits of policy distribution and critic value. Use shared backbone. You may use same architecture as in DQN task with one modification: instead of having a single output layer, it must have two output layers taking as input the output of the last hidden layer (one for actor, one for critic).

Still it may be very helpful to make more changes:
* use orthogonal initialization with gain $\sqrt{2}$ and initialize biases with zeros;
* use more filters (e.g. 32-64-64 instead of 16-32-64);
* use two-layer heads for actor and critic or add a linear layer into backbone;

**Danger:** do not divide on 255, input is already normalized to [0, 1] in our wrappers!

In [5]:
import torch
import torch.nn.functional as F
from torch import nn


class DQNAgent(nn.Module):
    def __init__(self, n_actions):

        super().__init__()
        self.n_actions = n_actions

        self.network = nn.Sequential(
            nn.Conv2d(4, 32, kernel_size=8, stride=4),
            nn.ReLU(),
            nn.Conv2d(32, 64, kernel_size=4, stride=2),
            nn.ReLU(),
            nn.Conv2d(64, 64, kernel_size=3, stride=1),
            nn.ReLU(),
            nn.Flatten(),
            nn.Linear(7 * 7 * 64, 512),
            nn.ReLU()
        )

        self.pi_head = nn.Sequential(
            nn.Linear(512, 512),
            nn.ReLU(),
            nn.Linear(512, self.n_actions)
        )

        self.value_head = nn.Sequential(
            nn.Linear(512, 512),
            nn.ReLU(),
            nn.Linear(512, 1)
        )

    def initialize_weights(m):
        if isinstance(m, nn.Conv2d):
            nn.init.orthogonal_(m.weight, np.sqrt(2))
            # if m.bias is not None:
            nn.init.zero_(m.bias)
        elif isinstance(m, nn.Linear):
            nn.init.zero_(m.bias)

    def forward(self, x):
        x = x / 255.0
        x = torch.Tensor(x)
        x = self.network(x)
        pi = self.pi_head(x)
        value = self.value_head(x)
        value = value.squeeze(1)
        return value, pi

You will also need to define and use a policy that wraps the model. While the model computes logits for all actions, the policy will sample actions and also compute their log probabilities.  `policy.act` should return a **dictionary** of all the arrays that are needed to interact with an environment and train the model.

**Important**: "actions" will be sent to environment, they must be numpy array or list, not PyTorch tensor.

Note: you can add more keys, e.g. it can be convenient to compute entropy right here.

In [16]:
class Policy:
    def __init__(self, model):
        self.model = model

    def act(self, inputs):
        inputs_tensor = torch.Tensor(inputs).to('cuda')
        values, logits = self.model(inputs_tensor)
        values, logits = values.cpu(), logits.cpu()
        probs = F.softmax(logits, dim=1)
        log_probs_all = F.log_softmax(logits, dim=1)
        probs_np = probs.detach().numpy()
        batch_size = probs_np.shape[0]
        chosen_actions = []
        for i in range(batch_size):
            chosen_actions.append(np.random.choice(probs_np.shape[1], p=probs_np[i]))
        chosen_actions = np.array(chosen_actions)
        selected_log_probs = log_probs_all[torch.arange(batch_size), torch.from_numpy(chosen_actions)]

        dct = {
            'values': values,
            'logits': logits.detach(),
            'log_probs': selected_log_probs,
            'probs': probs.detach(),
            'actions': chosen_actions
        }
        return dct


Next we will pass the environment and policy to a runner that collects rollouts from the environment.
The class is already implemented for you.

In [17]:
from runners import EnvRunner

This runner interacts with the environment for a given number of steps and returns a dictionary containing
keys

* 'observations'
* 'rewards'
* 'dones'
* 'actions'
* all other keys that you defined in `Policy`

under each of these keys there is a python `list` of interactions with the environment of specified length $T$ &mdash; the size of partial trajectory, or rollout length. Let's have a look at how it works.

In [18]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model = DQNAgent(n_actions=n_actions).to(device)
policy = Policy(model)
runner = EnvRunner(env, policy, nsteps=5)

In [19]:
# generates new rollout
trajectory = runner.get_next()

In [20]:
# what is inside
print(trajectory.keys())

dict_keys(['actions', 'values', 'logits', 'log_probs', 'probs', 'observations', 'rewards', 'dones'])


In [21]:
# Sanity checks
assert 'logits' in trajectory, "Not found: policy didn't provide logits"
assert 'log_probs' in trajectory, "Not found: policy didn't provide log_probs of selected actions"
assert 'values' in trajectory, "Not found: policy didn't provide critic estimations"
assert trajectory['logits'][0].shape == (nenvs, n_actions), "logits wrong shape"
assert trajectory['log_probs'][0].shape == (nenvs,), "log_probs wrong shape"
assert trajectory['values'][0].shape == (nenvs,), "values wrong shape"

for key in trajectory.keys():
    assert len(trajectory[key]) == 5, \
    f"something went wrong: 5 steps should have been done, got trajectory of length {len(trajectory[key])} for '{key}'"

Now let's work with this trajectory a bit. To train the critic you will need to compute the value targets. It will also be used as an estimation of $Q$ for actor training.

You should use all available rewards for value targets, so the formula for the value targets is simple:

$$
\hat v(s_t) = \sum_{t'=0}^{T - 1}\gamma^{t'}r_{t+t'} + \gamma^T \hat{v}(s_{t+T}),
$$

where $s_{t + T}$ is the latest observation of the environment.

Any callable could be passed to `EnvRunner` to be applied to each partial trajectory after it is collected.
Thus, we can implement and use `ComputeValueTargets` callable.

**Do not forget** to use `trajectory['dones']` flags to check if you need to add the value targets at the next step when
computing value targets for the current step.

**Bonus (+0.5 pts):** implement [Generalized Advantage Estimation (GAE)](https://arxiv.org/pdf/1506.02438.pdf) instead; use $\lambda \approx 0.95$ or even closer to 1 in experiment.

In [22]:
class ComputeValueTargets:
    def __init__(self, policy, gamma=0.99):
        self.policy = policy
        self.gamma = gamma

    def __call__(self, trajectory, latest_observation):
        v_critic = self.policy.act(latest_observation)['values']
        v_t = np.squeeze(v_critic.detach().cpu().numpy()).astype(np.float32)
        n_steps = len(trajectory['rewards'])
        value_targets = []
        for i in reversed(range(n_steps)):
            reward = np.array(trajectory['rewards'][i], dtype=np.float32)
            done = trajectory['dones'][i]
            mask = (np.array(done) == False).astype(np.float32)
            v_t = reward + self.gamma * v_t * mask
            value_targets.append(v_t)
        trajectory['value_targets'] = value_targets[::-1]

After computing value targets we will transform lists of interactions into tensors
with the first dimension `batch_size` which is equal to `T * nenvs`.

You need to make sure that after this transformation `"log_probs"`, `"value_targets"`, `"values"` are 1-dimensional PyTorch tensors.

In [23]:
class MergeTimeBatch:
    """ Merges first two axes (time and env batch). """
    def __call__(self, trajectory, latest_observation):
        env_steps = len(trajectory['observations'])
        nenvs = trajectory['actions'][0].shape[0]

        obs = np.array(trajectory['observations'])
        obs = obs.reshape((env_steps * nenvs,) + obs[0].shape[1:])
        trajectory['observations'] = torch.from_numpy(obs).float()

        for key in trajectory.keys():
            if key in ['values', 'logits', 'log_probs', 'probs']:
                trajectory[key] = torch.cat(trajectory[key], dim=0)
            elif key in ['actions']:
                trajectory[key] = torch.from_numpy(np.array(trajectory[key])).view(-1)
            elif key in ['rewards', 'resets', 'value_targets']:
                trajectory[key] = torch.from_numpy(np.array(trajectory[key], dtype=np.float32)).view(-1)

Let's do more sanity checks!

In [24]:
runner = EnvRunner(env, policy, nsteps=5, transforms=[ComputeValueTargets(policy),
                                                      MergeTimeBatch()])

trajectory = runner.get_next()

In [25]:
# More sanity checks
assert 'value_targets' in trajectory, "Value targets not found"
assert trajectory['log_probs'].shape == (5 * nenvs,)
assert trajectory['value_targets'].shape == (5 * nenvs,)
assert trajectory['values'].shape == (5 * nenvs,)

assert trajectory['log_probs'].requires_grad, "Gradients are not available for actor head!"
assert trajectory['values'].requires_grad, "Gradients are not available for critic head!"

Now is the time to implement the advantage actor critic algorithm itself. You can look into [Mnih et al. 2016](https://arxiv.org/abs/1602.01783) paper, and lectures ([part 1](https://www.youtube.com/watch?v=Ds1trXd6pos&list=PLkFD6_40KJIwhWJpGazJ9VSj9CFMkb79A&index=5), [part 2](https://www.youtube.com/watch?v=EKqxumCuAAY&list=PLkFD6_40KJIwhWJpGazJ9VSj9CFMkb79A&index=6)) by Sergey Levine.

In [27]:
from collections import defaultdict
from torch.nn.utils import clip_grad_norm_

In [28]:
def add_summary(env, key, value, step):
    env.writer.add_scalar(key, value, step)
    TensorboardSummaries._summaries.setdefault(key, []).append((step, value))


class A2C:
    def __init__(self,
                 policy,
                 optimizer,
                 value_loss_coef=0.25,
                 entropy_coef=0.01,
                 max_grad_norm=0.5):
        self.policy = policy
        self.optimizer = optimizer
        self.value_loss_coef = value_loss_coef
        self.entropy_coef = entropy_coef
        self.max_grad_norm = max_grad_norm

    @staticmethod
    def to_one_hot(y_tensor, ndims):
        """Helper: take an integer vector and convert it to a 1-hot matrix."""
        y_tensor = y_tensor.type(torch.LongTensor).view(-1, 1)
        y_one_hot = torch.zeros(y_tensor.size()[0], ndims).scatter_(1, y_tensor, 1)
        return y_one_hot

    def policy_loss(self, trajectory):
        log_probs_for_actions = trajectory['log_probs'].unsqueeze(1)
        advantages = trajectory['value_targets'] - trajectory['values'].detach()
        pi_loss = -torch.mean(log_probs_for_actions * advantages)
        self.advantages = torch.mean(advantages)
        return pi_loss

    def value_loss(self, trajectory):
        return F.mse_loss(trajectory['value_targets'], trajectory['values'])

    def loss(self, trajectory):
        pi_loss = self.policy_loss(trajectory)
        v_loss = self.value_loss(trajectory)
        log_probs_full = F.log_softmax(trajectory['logits'], dim=1)
        self.entropy = -torch.mean((trajectory['probs'] * log_probs_full).sum(1))
        main_loss = pi_loss + self.value_loss_coef * v_loss - self.entropy_coef * self.entropy
        return main_loss

    def step(self, trajectory):
        loss = self.loss(trajectory)
        loss.backward()
        self.grad_norm = nn.utils.clip_grad_norm_(self.policy.model.parameters(), self.max_grad_norm)
        self.optimizer.step()
        self.optimizer.zero_grad()

    def train(self, runner, num_epochs=10**7, summary_interval=50):
        """Trains the agent using rollouts from runner. Logs stats and plots summaries."""
        for i in range(num_epochs):
            runner.env.step_var = runner.step_var

            trajectory = runner.get_next()
            self.step(trajectory)

            add_summary(runner.env, 'loss', self.loss(trajectory).item(), runner.env.step_var)
            add_summary(runner.env, 'policy_loss', self.policy_loss(trajectory).item(), runner.env.step_var)
            add_summary(runner.env, 'value_loss', self.value_loss(trajectory).item(), runner.env.step_var)
            add_summary(runner.env, 'entropy', self.entropy.item(), runner.env.step_var)
            add_summary(runner.env, 'advantages', self.advantages.item(), runner.env.step_var)
            add_summary(runner.env, 'grad_norm', self.grad_norm.item(), runner.env.step_var)

            summary = ["SpaceInvadersNoFrameskip-v4/reward_mean_100",
                       "loss", "policy_loss", "value_loss", "entropy", "advantages", "grad_norm"]
            if i % summary_interval == 0:
                clear_output(wait=True)
                fig, axs = plt.subplots(nrows=2, ncols=4, figsize=(20, 10))
                for ax, name in zip(axs.ravel(), summary):
                    history = np.array(TensorboardSummaries._summaries.get(name, []))
                    if len(history) > 0:
                        print(f"{name}:\t{history[-1][1]}")
                        ax.set_title(name)
                        ax.set_xlabel("Steps")
                        ax.plot(history[:, 0], history[:, 1])
                        ax.grid()
                plt.show()


Now you can train your model. For optimization we suggest you use RMSProp with learning rate 7e-4 (you can also linearly decay it to 0), smoothing constant (alpha in PyTorch) equal to 0.99 and epsilon equal to 1e-5.

We recommend to train for at least 10 million environment steps across all batched environments (takes ~3 hours on a single GTX1080 with 8 CPU). It should be possible to achieve *average raw reward over last 100 episodes* (the average is taken over 100 last episodes in each environment in the batch) of about 600. **Your goal is to reach 500**.

Notes:
* if your reward is stuck at ~200 for more than 2M steps then probably there is a bug
* if your gradient norm is >10 something probably went wrong
* make sure your `entropy loss` is negative, your `critic loss` is positive
* make sure you didn't forget `.detach` in losses where it's needed
* `actor loss` should oscillate around zero or near it; do not expect loss to decrease in RL ;)
* you can experiment with `nsteps` ("rollout length"); standard rollout length is 5 or 10. Note that this parameter influences how many algorithm iterations is required to train on 10M steps (or 40M frames --- we used frameskip in preprocessing).

In [29]:
from IPython.display import clear_output
from collections import defaultdict
import matplotlib.pyplot as plt

In [30]:
import warnings
warnings.filterwarnings('ignore')

In [31]:
TensorboardSummaries._summaries = defaultdict(list)
optimizer = torch.optim.RMSprop(policy.model.parameters(), lr=7e-4, alpha=0.99, eps=1e-5)
a2c = A2C(policy, optimizer, value_loss_coef=0.25, entropy_coef=0.01, max_grad_norm=0.5)
num_epochs = 10**7

a2c.train(runner)

loss:	0.027711395174264908
policy_loss:	6.798788376727316e-08
value_loss:	0.1108459085226059
entropy:	1.5004972738097422e-05
advantages:	0.07129053771495819
grad_norm:	0.0675191879272461


KeyboardInterrupt: 

In [32]:
torch.save(model.state_dict(), "A2C")
env.close()

BrokenPipeError: [Errno 32] Broken pipe

## Evaluation

In [33]:
env = nature_dqn_env("SpaceInvadersNoFrameskip-v4", clip_reward=False, episodic_life=False)

In [34]:
def evaluate(env, policy, n_games=1, t_max=10000):
    '''
    Plays n_games and returns rewards
    '''
    rewards = []

    for _ in range(n_games):
        s, info = env.reset()

        R = 0
        for _ in range(t_max):
            action = policy.act(np.array([s]))["actions"][0]

            s, r, term, trank, _ = env.step(action)

            R += r
            if term or trank:
                break

        rewards.append(R)
    return np.array(rewards)

In [35]:
# evaluation will take some time!
sessions = evaluate(env, policy, n_games=30)
score = sessions.mean()
print(f"Your score: {score}")

assert score >= 500, "Needs more training?"
print("Well done!")

Your score: 285.0


AssertionError: Needs more training?

In [ ]:
env.close()

## Record

In [36]:
env_monitor = nature_dqn_env("SpaceInvadersNoFrameskip-v4", monitor=True, clip_reward=False, episodic_life=False)

In [37]:
# record sessions
sessions = evaluate(env_monitor, policy, n_games=3)

In [38]:
# rewards for recorded games
sessions

array([285., 285., 285.])

In [39]:
env_monitor.close()